In [8]:
# !pip install -q transformers einops accelerate langchain bitsandbytes langchain_community langchain_huggingface
# !pip install --upgrade transformers
# !pip install sentencepiece
%pip install langchain_huggingface
%pip install "numpy<2.0" 

  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.0
    Uninstalling numpy-2.1.0:
      Successfully uninstalled numpy-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.28.0 which is incompatible.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.26.4 which is incompatible.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# %pip install --upgrade accelerate einops 

In [1]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoConfig
import transformers
import torch

In [2]:
import urllib
import warnings
from pathlib import Path as p

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.document_loaders import PyPDFLoader

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
HF_TOKEN = os.environ['hf_token']
from huggingface_hub import login
login(HF_TOKEN)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /Users/mattsalomon/.cache/huggingface/token
Login successful


In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
#from langchain import HuggingFacePipeline

model = "meta-llama/Llama-2-7b-chat-hf"
#model = 'meta-llama/Meta-Llama-3.1-8B'
model='gpt2'

tokenizer = AutoTokenizer.from_pretrained(model,clean_up_tokenization_spaces = True)
tokenizer.pad_token = "[PAD]"
tokenizer.padding_side = "left"
pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    #max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    truncation=True,
    max_new_tokens=2000
)

/Users/mattsalomon/miniconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/mattsalomon/miniconda3/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <985F02DB-C495-332D-8ADD-45063184CA03> /Users/mattsalomon/miniconda3/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <EACD001F-FCB9-380E-AD73-D522177FC040> /Users/mattsalomon/miniconda3/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [6]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0.01})

In [11]:
openai_api_key=os.environ['OPENAI_API_KEY']
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    max_tokens=2000,
    openai_api_key=openai_api_key
)

In [24]:
# from langchain import  LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

# llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """ As part of Meta’s commitment to open science, today we are publicly releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational large language model designed to help researchers advance their work in this subfield of AI. Smaller, more performant models such as LLaMA enable others in the research community who don’t have access to large amounts of infrastructure to study these models, further democratizing access in this important, fast-changing field.

Training smaller foundation models like LLaMA is desirable in the large language model space because it requires far less computing power and resources to test new approaches, validate others’ work, and explore new use cases. Foundation models train on a large set of unlabeled data, which makes them ideal for fine-tuning for a variety of tasks. We are making LLaMA available at several sizes (7B, 13B, 33B, and 65B parameters) and also sharing a LLaMA model card that details how we built the model in keeping with our approach to Responsible AI practices.

Over the last year, large language models — natural language processing (NLP) systems with billions of parameters — have shown new capabilities to generate creative text, solve mathematical theorems, predict protein structures, answer reading comprehension questions, and more. They are one of the clearest cases of the substantial potential benefits AI can offer at scale to billions of people.

Even with all the recent advancements in large language models, full research access to them remains limited because of the resources that are required to train and run such large models. This restricted access has limited researchers’ ability to understand how and why these large language models work, hindering progress on efforts to improve their robustness and mitigate known issues, such as bias, toxicity, and the potential for generating misinformation.

Smaller models trained on more tokens — which are pieces of words — are easier to retrain and fine-tune for specific potential product use cases. We trained LLaMA 65B and LLaMA 33B on 1.4 trillion tokens. Our smallest model, LLaMA 7B, is trained on one trillion tokens.

Like other large language models, LLaMA works by taking a sequence of words as an input and predicts a next word to recursively generate text. To train our model, we chose text from the 20 languages with the most speakers, focusing on those with Latin and Cyrillic alphabets.

There is still more research that needs to be done to address the risks of bias, toxic comments, and hallucinations in large language models. Like other models, LLaMA shares these challenges. As a foundation model, LLaMA is designed to be versatile and can be applied to many different use cases, versus a fine-tuned model that is designed for a specific task. By sharing the code for LLaMA, other researchers can more easily test new approaches to limiting or eliminating these problems in large language models. We also provide in the paper a set of evaluations on benchmarks evaluating model biases and toxicity to show the model’s limitations and to support further research in this crucial area.

To maintain integrity and prevent misuse, we are releasing our model under a noncommercial license focused on research use cases. Access to the model will be granted on a case-by-case basis to academic researchers; those affiliated with organizations in government, civil society, and academia; and industry research laboratories around the world. People interested in applying for access can find the link to the application in our research paper.

We believe that the entire AI community — academic researchers, civil society, policymakers, and industry — must work together to develop clear guidelines around responsible AI in general and responsible large language models in particular. We look forward to seeing what the community can learn — and eventually build — using LLaMA.
"""
translating_prompt_template = """
translate {summary} to french:
"""

translating_prompt = PromptTemplate.from_template(translating_prompt_template)
translating_chain = translating_prompt | llm
summarizing_chain = prompt | llm | StrOutputParser()

overall_chain = summarizing_chain | {
    'summary': summarizing_chain,
    'translation': translating_chain
}

result = overall_chain.invoke(text)


In [25]:
print(result['summary'])

- Meta has released LLaMA, a large language model, for AI researchers
- LLaMA is smaller and requires less computing power, making it more accessible
- Available in different sizes with a model card detailing its construction
- Large language models have shown capabilities but are limited due to resource requirements
- LLaMA can be retrained for specific use cases and predicts the next word in a sequence
- More research is needed to address bias and toxicity in large language models
- LLaMA is released under a noncommercial license for research purposes
- Collaboration in the AI community is needed to develop guidelines for responsible AI, including large language models


In [26]:
print(result['translation'])

content="Meta a publiquement publié LLaMA, un grand modèle de langage, pour aider les chercheurs dans le domaine de l'IA\n- Les modèles plus petits comme LLaMA nécessitent moins de puissance de calcul et de ressources, les rendant plus accessibles aux chercheurs\n- LLaMA est disponible en différentes tailles et une carte de modèle détaillant sa construction a été partagée\n- Les grands modèles de langage ont montré des capacités dans diverses tâches mais l'accès à eux est limité en raison des exigences en ressources\n- LLaMA est entraîné sur un grand ensemble de jetons et peut être retravaillé pour des cas d'utilisation spécifiques\n- Le modèle fonctionne en prédisant le mot suivant dans une séquence et est conçu pour être polyvalent pour différents cas d'utilisation\n- Plus de recherche est nécessaire pour aborder les biais, la toxicité et d'autres problèmes dans les grands modèles de langage\n- LLaMA est publié sous une licence non commerciale à des fins de recherche et l'accès est a

In [14]:
text1 = """
Tesla, Inc. (/ˈtɛslə/ TESS-lə or /ˈtɛzlə/ TEZ-lə[a]) is an American multinational automotive and clean energy company headquartered in Austin, Texas. Tesla designs and manufactures electric vehicles (cars and trucks), stationary battery energy storage devices from home to grid-scale, solar panels and solar roof tiles, and related products and services.

Tesla is one of the world's most valuable companies and, as of 2023, was the world's most valuable automaker. In 2022, the company led the battery electric vehicle market, with 18% share.

Its subsidiary Tesla Energy develops and is a major installer of photovoltaic systems in the United States. Tesla Energy is one of the largest global suppliers of battery energy storage systems, with 6.5 gigawatt-hours (GWh) installed in 2022.

Tesla was incorporated in July 2003 by Martin Eberhard and Marc Tarpenning as Tesla Motors. The company's name is a tribute to inventor and electrical engineer Nikola Tesla. In February 2004, via a $6.5 million investment, Elon Musk became the company's largest shareholder. He became CEO in 2008. Tesla's announced mission is to help expedite the move to sustainable transport and energy, obtained through electric vehicles and solar power.

Tesla began production of its first car model, the Roadster sports car, in 2008. This was followed by the Model S sedan in 2012, the Model X SUV in 2015, the Model 3 sedan in 2017, the Model Y crossover in 2020, and the Tesla Semi truck in 2022. The company plans production of the Cybertruck light-duty pickup truck in 2023.[8] The Model 3 is the all-time bestselling plug-in electric car worldwide, and in June 2021 became the first electric car to sell 1 million units globally.[9] Tesla's 2022 deliveries were around 1.31 million vehicles, a 40% increase over the previous year,[10][11] and cumulative sales totaled 4 million cars as of April 2023.[12] In October 2021, Tesla's market capitalization temporarily reached $1 trillion, the sixth company to do so in U.S. history.

Tesla has been the subject of lawsuits, government scrutiny, and journalistic criticism, stemming from allegations of whistleblower retaliation, worker rights violations, product defects, and Musk's many controversial statements.
"""

result = overall_chain.invoke(text1)

In [15]:
print(result['summary'])

- Tesla, Inc. is an American multinational automotive and clean energy company headquartered in Austin, Texas
- Tesla designs and manufactures electric vehicles, stationary battery energy storage devices, solar panels, and related products
- Tesla is one of the world's most valuable companies and the most valuable automaker as of 2023
- Tesla Energy is a major installer of photovoltaic systems and a global supplier of battery energy storage systems
- Tesla was founded in 2003 by Martin Eberhard and Marc Tarpenning, with Elon Musk becoming the largest shareholder in 2004 and CEO in 2008
- Tesla's mission is to expedite the move to sustainable transport and energy through electric vehicles and solar power
- Tesla has produced several car models, with the Model 3 being the all-time bestselling plug-in electric car worldwide
- Tesla's 2022 deliveries were around 1.31 million vehicles, with cumulative sales totaling 4 million cars as of April 2023
- Tesla's market capitalization temporarily

In [16]:
text2 ="""
Apple Inc. is an American multinational technology company headquartered in Cupertino, California. Apple is the world's largest technology company by revenue, with US$394.3 billion in 2022 revenue.[6] As of March 2023, Apple is the world's biggest company by market capitalization.[7] As of June 2022, Apple is the fourth-largest personal computer vendor by unit sales and the second-largest mobile phone manufacturer in the world. It is often considered as one of the Big Five American information technology companies, alongside Alphabet (parent company of Google), Amazon, Meta Platforms, and Microsoft.

Apple was founded as Apple Computer Company on April 1, 1976, by Steve Wozniak, Steve Jobs (1955–2011) and Ronald Wayne to develop and sell Wozniak's Apple I personal computer. It was incorporated by Jobs and Wozniak as Apple Computer, Inc. in 1977. The company's second computer, the Apple II, became a best seller and one of the first mass-produced microcomputers. Apple went public in 1980 to instant financial success. The company developed computers featuring innovative graphical user interfaces, including the 1984 original Macintosh, announced that year in a critically acclaimed advertisement called "1984". By 1985, the high cost of its products, and power struggles between executives, caused problems. Wozniak stepped back from Apple and pursued other ventures, while Jobs resigned and founded NeXT, taking some Apple employees with him.

As the market for personal computers expanded and evolved throughout the 1990s, Apple lost considerable market share to the lower-priced duopoly of the Microsoft Windows operating system on Intel-powered PC clones (also known as "Wintel"). In 1997, weeks away from bankruptcy, the company bought NeXT to resolve Apple's unsuccessful operating system strategy and entice Jobs back to the company. Over the next decade, Jobs guided Apple back to profitability through a number of tactics including introducing the iMac, iPod, iPhone and iPad to critical acclaim, launching the "Think different" campaign and other memorable advertising campaigns, opening the Apple Store retail chain, and acquiring numerous companies to broaden the company's product portfolio. When Jobs resigned in 2011 for health reasons, and died two months later, he was succeeded as CEO by Tim Cook.

Apple became the first publicly traded U.S. company to be valued at over $1 trillion in August 2018, then at $2 trillion in August 2020, and at $3 trillion in January 2022. As of April 2023, it was valued at around $2.6 trillion. The company receives criticism regarding the labor practices of its contractors, its environmental practices, and its business ethics, including anti-competitive practices and materials sourcing. Nevertheless, the company has a large following and enjoys a high level of brand loyalty. It has also been consistently ranked as one of the world's most valuable brands.
"""
result = overall_chain.invoke(text2)

In [17]:
print(result['summary'])

- Apple Inc. is an American multinational technology company based in Cupertino, California
- Founded in 1976 by Steve Wozniak, Steve Jobs, and Ronald Wayne
- Steve Jobs returned in 1997 and led the company to profitability with innovative products
- Tim Cook became CEO in 2011
- Apple reached $1 trillion, $2 trillion, and $3 trillion valuations
- Faces criticism for labor practices, environmental practices, and business ethics
- Despite criticism, Apple has a large following, high brand loyalty, and is consistently ranked as one of the world's most valuable brands


In [18]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
## Splittting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])
len(chunks)

5

In [19]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
summary = chain.run(chunks)

In [20]:
summary

'Meta has released LLaMA, a large language model, to support open science and make advanced AI research more accessible. LLaMA is a smaller, more efficient alternative to large language models, making it easier to fine-tune for specific tasks. The model is trained on text from 20 languages and faces challenges such as bias and toxicity. Access to LLaMA is limited to researchers for noncommercial use to promote responsible AI development and collaboration.'

In [21]:
template = """
              Detect named entities in following text delimited by triple backquotes.
              Return your response in json format with spans of named entities with fields "named entity","type","span".
              Return all entities
              ```{text}```
              json format file:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

#llm_chain = LLMChain(prompt=prompt, llm=llm)
summarizing_chain = prompt | llm | StrOutputParser()
overall_chain = summarizing_chain | {
    'summary': summarizing_chain,
    'translation': translating_chain
}

In [22]:
# print(llm_chain.run(text2))
result = overall_chain.invoke(text2)
print(result)

{'summary': '{\n    "entities": [\n        {\n            "named entity": "Apple Inc.",\n            "type": "ORG",\n            "span": "0-9"\n        },\n        {\n            "named entity": "American",\n            "type": "MISC",\n            "span": "13-21"\n        },\n        {\n            "named entity": "Cupertino",\n            "type": "LOC",\n            "span": "63-71"\n        },\n        {\n            "named entity": "California",\n            "type": "LOC",\n            "span": "73-83"\n        },\n        {\n            "named entity": "Apple",\n            "type": "ORG",\n            "span": "85-90"\n        },\n        {\n            "named entity": "US$394.3 billion",\n            "type": "MONEY",\n            "span": "107-122"\n        },\n        {\n            "named entity": "March 2023",\n            "type": "DATE",\n            "span": "144-153"\n        },\n        {\n            "named entity": "June 2022",\n            "type": "DATE",\n            "span"

In [23]:
result = overall_chain.invoke(text1)
print(result)

{'summary': '{\n    "entities": [\n        {\n            "named entity": "Tesla, Inc.",\n            "type": "ORG",\n            "span": "Tesla, Inc."\n        },\n        {\n            "named entity": "American",\n            "type": "MISC",\n            "span": "American"\n        },\n        {\n            "named entity": "Austin",\n            "type": "LOC",\n            "span": "Austin"\n        },\n        {\n            "named entity": "Texas",\n            "type": "LOC",\n            "span": "Texas"\n        },\n        {\n            "named entity": "Tesla",\n            "type": "ORG",\n            "span": "Tesla"\n        },\n        {\n            "named entity": "2023",\n            "type": "DATE",\n            "span": "2023"\n        },\n        {\n            "named entity": "2022",\n            "type": "DATE",\n            "span": "2022"\n        },\n        {\n            "named entity": "United States",\n            "type": "LOC",\n            "span": "United States"